In [6]:
import numpy as np
import json
from cad_viewer_widget import CadViewer

In [7]:
box1 = {
    "name": "Group",
    "id": "/Group",
    "loc": None,
    "parts": [
        {
            "type": "shapes",
            "id": "/Group/Part_0",            
            "name": "Part_0",
            "color": "#e8b024",            
            "shape": {
                "vertices": np.asarray([
                    [-0.5, -0.5, -0.5],
                    [-0.5, -0.5, 0.5],
                    [-0.5, 0.5, -0.5],
                    [-0.5, 0.5, 0.5],
                    [0.5, -0.5, -0.5],
                    [0.5, -0.5, 0.5],
                    [0.5, 0.5, -0.5],
                    [0.5, 0.5, 0.5],
                    [-0.5, -0.5, -0.5],
                    [0.5, -0.5, -0.5],
                    [-0.5, -0.5, 0.5],
                    [0.5, -0.5, 0.5],
                    [-0.5, 0.5, -0.5],
                    [0.5, 0.5, -0.5],
                    [-0.5, 0.5, 0.5],
                    [0.5, 0.5, 0.5],
                    [-0.5, -0.5, -0.5],
                    [-0.5, 0.5, -0.5],
                    [0.5, -0.5, -0.5],
                    [0.5, 0.5, -0.5],
                    [-0.5, -0.5, 0.5],
                    [-0.5, 0.5, 0.5],
                    [0.5, -0.5, 0.5],
                    [0.5, 0.5, 0.5]
                ]),
                "triangles": [
                    1, 2, 0, 
                    1, 3, 2, 
                    5, 4, 6, 
                    5, 6, 7, 
                    11, 8, 9, 
                    11, 10, 8, 
                    15, 13, 12, 
                    15, 12, 14, 
                    19, 16, 17, 
                    19, 18, 16, 
                    23, 21, 20, 
                    23, 20, 22],
                "normals": np.asarray([
                    [-1.0, -0.0, 0.0],
                    [-1.0, -0.0, 0.0],
                    [-1.0, -0.0, 0.0],
                    [-1.0, -0.0, 0.0],
                    [1.0, 0.0, -0.0],
                    [1.0, 0.0, -0.0],
                    [1.0, 0.0, -0.0],
                    [1.0, 0.0, -0.0],
                    [-0.0, -1.0, -0.0],
                    [-0.0, -1.0, -0.0],
                    [-0.0, -1.0, -0.0],
                    [-0.0, -1.0, -0.0],
                    [0.0, 1.0, 0.0],
                    [0.0, 1.0, 0.0],
                    [0.0, 1.0, 0.0],
                    [0.0, 1.0, 0.0],
                    [-0.0, -0.0, -1.0],
                    [-0.0, -0.0, -1.0],
                    [-0.0, -0.0, -1.0],
                    [-0.0, -0.0, -1.0],
                    [0.0, 0.0, 1.0],
                    [0.0, 0.0, 1.0],
                    [0.0, 0.0, 1.0],
                    [0.0, 0.0, 1.0]
                ]),
                "edges": np.asarray([
                        [[-0.5, -0.5, -0.5], [-0.5, -0.5, 0.5]],
                        [[-0.5, -0.5, 0.5], [-0.5, 0.5, 0.5]],
                        [[-0.5, 0.5, -0.5], [-0.5, 0.5, 0.5]],
                        [[-0.5, -0.5, -0.5], [-0.5, 0.5, -0.5]],
                        [[0.5, -0.5, -0.5], [0.5, -0.5, 0.5]],
                        [[0.5, -0.5, 0.5], [0.5, 0.5, 0.5]],
                        [[0.5, 0.5, -0.5], [0.5, 0.5, 0.5]],
                        [[0.5, -0.5, -0.5], [0.5, 0.5, -0.5]],
                        [[-0.5, -0.5, -0.5], [0.5, -0.5, -0.5]],
                        [[-0.5, -0.5, 0.5], [0.5, -0.5, 0.5]],
                        [[-0.5, 0.5, -0.5], [0.5, 0.5, -0.5]],
                        [[-0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]
                ])
            }
        }
    ]
}


states = { "/Group/Part_0":[1,1] }

In [8]:
cv = CadViewer(options={"cadWidth":640, "height":480, "theme":"light"})
cv.add_shapes(box1, states, options={})

CadViewerWidget(options={'cadWidth': 640, 'height': 480, 'theme': 'light', 'treeWidth': 240, 'needsAnimationLo…

In [9]:
cv.add_shapes(box1, states, options={})

In [10]:
result = cv.execute("viewer.camera.position", "set", (1, 1, .1), None, True)
result

{'msg_id': 1, 'result': {}}

In [12]:
print(result["msg_id"])
print(cv.results[result["msg_id"]])

1
{'x': 0.9999999999999997, 'y': 0.9999999999999996, 'z': 0.09999999999999992}


In [13]:
cv.execute("viewer.camera.zoom", "=", 0.8, None, True)

{'msg_id': 2,
 'result': {1: {'x': 0.9999999999999997,
   'y': 0.9999999999999996,
   'z': 0.09999999999999992}}}

In [14]:
cv.execute("viewer", "switchCamera", False)

{'msg_id': 3,
 'result': {1: {'x': 0.9999999999999997,
   'y': 0.9999999999999996,
   'z': 0.09999999999999992},
  2: [0.8]}}